CS524: Introduction to Optimization Lecture 9
======================================

## Michael Ferris<br> Computer Sciences Department <br> University of Wisconsin-Madison

## September 23, 2024
--------------

# Assigning students to projects to maximize overall satisfaction

- A teacher wishes to assign 5 projects to 5 different students. 
- Each student has indicated their preference for each project by assigning it a score between 0 and 10 
- (0 indicating strong dislike and 10 indicating strong preference)
- The teacher wishes to make the assignment of projects to students in a way that maximizes their overall satisfaction, as measured by the sum of the preferences for the given assignments.

In [1]:
import sys
import numpy as np

from gamspy import (
    Container,Set,Alias,Parameter,Variable,Equation,Model,Problem,Sense,Options,
    Domain,Number,Sum,Product,Smax,Smin,Ord,Card,SpecialValues,
)

m = Container()

In [2]:
# SETS #
projects = Set(m,'projects',records=[f"proj{i}" for i in range(1,6)])
students = Set(m,'students',records=[f"student{i}" for i in range(1,6)])
preferences = Parameter(m,'preferences',[students,projects],records=np.array(
    [   [ 7,6,5,8,9],
        [ 0,3,0,8,5],
        [ 0,0,0,4,3],
        [ 0,9,3,0,9],
        [ 0,6,7,6,0]] ))

display(preferences.pivot())
# VARIABLES #
# want x to be 1 if that student is assigned to that project; 0 otherwise
# (actually in this simple assignment problem, dont need the variables to be binary)

x = Variable(m,"x","binary",domain=[students,projects])

# EQUATIONS #

EQassignProjects = Equation(m,'EQassignProjects',domain=[projects],description="ensure that each project is assigned to one of the student")  
EQassignProjects[projects]= Sum(students, x[students,projects]) == 1 

EQassignStudents = Equation(m,'EQassignStudents',domain=[students],description="ensure that each students is assigned exactly one project")  
EQassignStudents[students]=  Sum(projects, x[students,projects]) == 1

happyclass = Model(m,
    name="happyclass",
    equations=m.getEquations(),
    problem=Problem.MIP,
    sense=Sense.MAX,
    objective= Sum([students,projects],
        preferences[students,projects] * x[students,projects]),
)

happyclass.solve()

,proj1,proj2,proj3,proj4,proj5
student1,7.0,6.0,5.0,8.0,9.0
student2,0.0,3.0,0.0,8.0,5.0
student3,0.0,0.0,0.0,4.0,3.0
student4,0.0,9.0,3.0,0.0,9.0
student5,0.0,6.0,7.0,6.0,0.0


,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,34,11,26,MIP,CPLEX,0.042


In [3]:
print("Objective Function Value:  ", round(happyclass.objective_value, 4), "\n")
matchings =  x.records[["students","projects"]][x.records["level"] > 0.0]
display("x:  ", matchings)

Objective Function Value:   34.0 



'x:  '

,students,projects
0,student1,proj1
8,student2,proj4
14,student3,proj5
16,student4,proj2
22,student5,proj3


# Can switch to relaxed mip (replace binary variables with [0,1] variables and solve LP with TU matrix

In [4]:
happyclass = Model(m,
    name="happyclass",
    equations=m.getEquations(),
    problem=Problem.RMIP,
    sense=Sense.MAX,
    objective= Sum([students,projects],
        preferences[students,projects] * x[students,projects]),
)
   
happyclass.solve(solver='cplex',
    solver_options={'lpmethod': 3, 'netfind': 2, 'preind': 0})

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,34,11,26,RMIP,CPLEX,0.011


In [5]:
print("Objective Function Value:  ", round(happyclass.objective_value, 4), "\n")
matchings =  x.records[["students","projects","level"]][x.records["level"] > 0.0]
display("x:  ", matchings)

Objective Function Value:   34.0 



'x:  '

,students,projects,level
0,student1,proj1,1.0
8,student2,proj4,1.0
14,student3,proj5,1.0
16,student4,proj2,1.0
22,student5,proj3,1.0


## Can we also maximize the minimium assigned preference?

In [6]:
prefgot = Parameter(m,'prefgot',domain=['*',students])
prefgot['happyclass',students] = Sum(projects, x.l[students,projects] * preferences[students,projects])

minpref = Parameter(m,'minpref', domain=[students])
minpref[students] = Smin(projects, preferences[students, projects]+SpecialValues.EPS) 

In [7]:
z = Variable(m,"z","free")
zdef = Equation(m,"zdef",domain=[students])
zdef[students]= z <= Sum(projects, preferences[students,projects] * x[students,projects])

minmax = Model(m,
    name="minmax",
    equations=[EQassignProjects, EQassignStudents, zdef],
    problem=Problem.RMIP,
    sense=Sense.MAX,
    objective=z,
)

minmax.solve(solver='cplex',solver_options={'lpmethod': 3, 'netfind': 2, 'preind': 0},output=None)

prefgot['minmax(rmip)',students] = Sum(projects, x.l[students,projects] * preferences[students,projects])

minmax = Model(m,
    name="minmax",
    equations=[EQassignProjects, EQassignStudents, zdef],
    problem=Problem.MIP,
    sense=Sense.MAX,
    objective=z,
)

minmax.solve(solver='cplex',solver_options={'lpmethod': 3, 'netfind': 2, 'preind': 0})

prefgot['minmax(mip)',students] = Sum(projects, x.l[students,projects] * preferences[students,projects])

In [8]:
display("Min preference:",minpref.records, "Preference got:",prefgot.records)

'Min preference:'

,students,value
0,student1,5.0
1,student2,-0.0
2,student3,-0.0
3,student4,-0.0
4,student5,-0.0


'Preference got:'

,uni,students,value
0,happyclass,student1,7.0
1,happyclass,student2,8.0
2,happyclass,student3,3.0
3,happyclass,student4,9.0
4,happyclass,student5,7.0
5,minmax(rmip),student1,7.0
6,minmax(rmip),student2,5.0
7,minmax(rmip),student3,4.0
8,minmax(rmip),student4,9.0
9,minmax(rmip),student5,7.0


# Now do this as a MCNF

In [12]:
# Convert data into MCNF data
node = Set(m,'node',domain=['*'])
node[projects] = True
node[students] = True

arc = Set(m,'arc',domain=['*','*'])
arc[students,projects] = True

i = Alias(m,'i',node)
j = Alias(m,'j',node)
k = Alias(m,'k',node)

b = Parameter(m,'b',domain=['*'])
b[projects] = -1
b[students] = 1

c = Parameter(m,'c',domain=['*','*'])
c[students,projects] = preferences[students,projects]

# VARIABLES #
f = Variable(m,"f","binary",domain=['*','*'],description="flow")

# EQUATIONS #
# ensure flow out - flow in = b, use dynamic set 
balance = Equation(m,'balance',domain=['*'])
balance[i]= Sum(arc[i,k], f[i,k]) - Sum(arc[j,i], f[j,i]) == b[i]

mcf = Model(m,
    name="mcf",
    equations=[balance],
    problem=Problem.RMIP,
    sense=Sense.MAX,
    objective=Sum(arc[i,j], c[i,j]*f[i,j]),
)

mcf.solve(solver='cplex',solver_options={'lpmethod': 3, 'netfind': 2, 'preind': 0, 'names': 'no'},output=None)

print("Objective Function Value:  ", round(mcf.objective_value, 4), "\n\nf:")
display(f.pivot())

Exception: Pivoting operations only possible on symbols with dimension > 1, symbol dimension is 1